<img src="assets/banner2.jpg"> </img>
<h1 style="text-align:center">Creación de pipeline con modelo preentrenado</h1>


In [2]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start()

print(f"Versión de spark: {spark.version}")
print(f"Versión de sparknlp: {sparknlp.version()}")

Versión de spark: 3.1.1
Versión de sparknlp: 3.0.1


#### Creando un pipeline de analisis de sentimientos utilizando un pipeline preentrenado

Primero formateamos el dataframe para obtener los textos con lo que vamos a probar el pipeline preentrenado.

In [26]:
example_data = dataframe.select("text").limit(5)

In [27]:
SENTIMENT_MODEL_NAME='classifierdl_use_emotion'

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler,
        use,
        sentimentdl
])


tfhub_use download started this may take some time.
Approximate size to download 923,7 MB
[OK!]


KeyboardInterrupt: 

In [ ]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

In [25]:
result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment")).show(truncate=False)

+--------------------+
|                text|
+--------------------+
|@VirginAmerica Wh...|
|@VirginAmerica pl...|
|@VirginAmerica I ...|
|"@VirginAmerica i...|
|@VirginAmerica an...|
+--------------------+

